In [1]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import tools
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.3)

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
inputVideo =  "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker208/speaker208_0000.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
tools.splitVideo(inputVideo,2,videoDuration)

In [3]:
#Carga vídeo
res = tools.saveFaceCrops(inputVideo,detector)
res[0].shape

(112, 112)

In [8]:
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
#_,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker184_0004.wav") 
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR


#TEST
#audio = torch.FloatTensor(audio[:180]).unsqueeze(0)
#video = torch.FloatTensor(res[:45]).unsqueeze(0)
center = 23
audio = tools.padAudio(audio,1,center,51,len(res))
video = tools.padVideo(res,center,51)
print("audio shape: ", audio.shape)
print("video shape: ", video.shape)

904
226
audio shape:  torch.Size([204, 13])
video shape:  torch.Size([51, 112, 112])


In [5]:
model = talkNet()
model.load_state_dict(torch.load("./exps/exp2/model/model_0003.model"))

05-23 11:10:19 Model para number = 15.01


<All keys matched successfully>

In [6]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(1050)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [7]:
correctFrames, totalFrames = 0, 0
correctSamples, totalSamples = 0, 0
totalPreds = []
for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
    with torch.no_grad():    
        output = model((audioFeature,visualFeature))
        labels = labels.cuda()
        batchPreds = torch.reshape(output[1], labels.shape)
        #Precision a nivel de video
        videoPreds = torch.mode(batchPreds,dim=1)[0]
        totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
        labelMode = torch.mode(labels,dim=1)[0]
        correctSamples += (videoPreds == labelMode).sum().float()
        totalSamples += len(labels)
        # Precision a nivel de frame
        labels = labels.reshape((-1))
        correctFrames += (output[1] == labels).sum().float()
        totalFrames += len(labels) 
#7865
print("Frame acc:", correctFrames/totalFrames)
print("Sample acc:", correctSamples/totalSamples)

  0%|          | 0/938 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print("Audios iguales?",(audio == item[0]).sum(),audio.shape[0]*audio.shape[1])
print("Videos iguales?",(video == item[1]).sum(),video.shape[0]*112*112)
print(audio[0],item[0][0])

Audios iguales? tensor(2652) 2652
Videos iguales? tensor(328442) 639744
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [9]:
iAudio = audio.unsqueeze(0)
iVideo = video.unsqueeze(0)
# iAudio = item[0].unsqueeze(0)
# iVideo = item[1].unsqueeze(0)
output = model((iAudio,iVideo))
#output = model((audio,video))
print(output)
print("Pred ->",torch.mode(output[1])[0])

(tensor([[0.0040, 0.9960],
        [0.0023, 0.9977],
        [0.0020, 0.9980],
        [0.0026, 0.9974],
        [0.0058, 0.9942],
        [0.0054, 0.9946],
        [0.0031, 0.9969],
        [0.0020, 0.9980],
        [0.0081, 0.9919],
        [0.0055, 0.9945],
        [0.0066, 0.9934],
        [0.0085, 0.9915],
        [0.0030, 0.9970],
        [0.0084, 0.9916],
        [0.0070, 0.9930],
        [0.0056, 0.9944],
        [0.0061, 0.9939],
        [0.0083, 0.9917],
        [0.0542, 0.9458],
        [0.0507, 0.9493],
        [0.0072, 0.9928],
        [0.0051, 0.9949],
        [0.0061, 0.9939],
        [0.0094, 0.9906],
        [0.0171, 0.9829],
        [0.0101, 0.9899],
        [0.0037, 0.9963],
        [0.0129, 0.9871],
        [0.0034, 0.9966],
        [0.0345, 0.9655],
        [0.0248, 0.9752],
        [0.0069, 0.9931],
        [0.0077, 0.9923],
        [0.0058, 0.9942],
        [0.0120, 0.9880],
        [0.0121, 0.9879],
        [0.0026, 0.9974],
        [0.0024, 0.9976],
        [0.

In [ ]:
#print(totalPreds)
dfPreds = pd.DataFrame(totalPreds,columns=["Prediction"])
dfSamples = pd.read_csv("testSamples.csv")
dfRes = dfSamples.join(dfPreds)
print(dfRes)
dfRes.to_csv("testPreds5.csv")

       Unnamed: 0            video            audio  label  center  Prediction
0               0  speaker073_0000  speaker073_0000      1      34           1
1               1  speaker205_0000  speaker205_0001      0      75           0
2               2  speaker163_0000  speaker163_0000      1     118           1
3               3  speaker314_0011  speaker314_0011      1      18           1
4               4  speaker300_0001  speaker300_0001      0      19           0
...           ...              ...              ...    ...     ...         ...
29996       29996  speaker310_0006  speaker310_0006      1      59           1
29997       29997  speaker169_0002  speaker169_0002      1      50           1
29998       29998  speaker291_0000  speaker155_0006      0     143           0
29999       29999  speaker258_0000  speaker197_0005      0      96           0
30000       30000  speaker065_0025  speaker065_0025      1      25           1

[30001 rows x 6 columns]


In [ ]:
dfTrain = pd.read_csv("trainSamples.csv")

In [ ]:
dfTrain.join(dfSamples,on="video")

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat